# Python Machine Learning library: XGBoost

## 0. Some recap


Supervised learning problem: classification, regression, ranking, recommendation.     

Classification problem:     

When dealing with binary supervised learning problems, the AUC (Area under the Receiver Operating Characteristic (ROC)) is the most versatile and common evaluation metric used to judge the quality of a binary classification model. It is simply the probability that a randomly chosen positive data point will have a higher rank than a randomly chosen negative data point for the learning problem. Higher AUC = more sensitive, better performing model.      

When dealing with multi-class classification problem, it is common to use accuracy score (higher is better) or to look at overall confusion matrix to evaluate the quality of a model.     




## 1. XGBoost Introduction

Optimised gradient-boosting machine learning library.      
Has API in several language: Python, R etc.       
Speed and performance is its key.       
Core XGBoost algorithm is parallelisable, it can harness all of the processing power of modern multi-core computers.     
Consistently outperforms single-algorithm methods in ML competition and has been shown to achieve state of the art performance on a variety of benchmark ML dataset.      


In [2]:
#quick example
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split